In [6]:
"""Heatmap."""

import matplotlib.pyplot as plt
from scipy.sparse import load_npz
import pandas as pd

In [10]:
image_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-wsi-img.tiff'
cell_gene_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-gene.npz'
cell_info_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell-info.csv'

In [11]:
cell_centroids = pd.read_csv(cell_info_path)
cell_centroids[['x', 'y']].values

In [15]:
cell_centroids[['x', 'y']].values

48557

In [4]:
load_npz(cell_gene_path)

<48557x3059 sparse matrix of type '<class 'numpy.float32'>'
	with 32269268 stored elements in Compressed Sparse Row format>